In [10]:
import numpy as np
from numpy.fft import fft2, ifft2, fftshift
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Define parameters for the simulation
nx, ny = 100, 100  # Grid size
dx, dy = 1.0, 1.0  # Differential step size
dt = 0.01  # Time step size
nt = 600  # Number of time steps to include in the animation

# Define the wave numbers for the initial condition
kx = 2 * np.pi / (nx * dx)  # Wave number in x-direction
ky = 2 * np.pi / (ny * dy)  # Wave number in y-direction

# Initialize the vorticity with a sinusoidal function
X, Y = np.meshgrid(np.arange(nx), np.arange(ny))
omega = np.sin(kx * X) * np.sin(ky * Y)

# Define the forward Euler update function
def forward_euler(omega, u, v, nx, ny, dx, dy, dt):
    omega_new = np.zeros_like(omega)
    for i in range(nx):
        for j in range(ny):
            dwdx = (omega[(i+1)%nx, j] - omega[(i-1)%nx, j]) / (2*dx)
            dwdy = (omega[i, (j+1)%ny] - omega[i, (j-1)%ny]) / (2*dy)
            omega_new[i, j] = omega[i, j] - dt * (u[i, j] * dwdx + v[i, j] * dwdy)
    return omega_new

# Solve the Poisson equation for the stream function using FFT
def solve_poisson_fft(omega, dx, dy):
    kx = 2 * np.pi * np.fft.fftfreq(nx, d=dx)
    ky = 2 * np.pi * np.fft.fftfreq(ny, d=dy)
    Kx, Ky = np.meshgrid(kx, ky)
    psi_hat = -fft2(omega) / (Kx**2 + Ky**2 + 1e-10)  # Avoid division by zero
    psi_hat[0, 0] = 0  # Zero out the mean flow component
    psi = np.real(ifft2(psi_hat))
    return psi

# Calculate the velocity field from the stream function
def velocity_from_streamfunction(psi, dx, dy):
    u = (np.roll(psi, -1, axis=0) - np.roll(psi, 1, axis=0)) / (2 * dy)
    v = -(np.roll(psi, -1, axis=1) - np.roll(psi, 1, axis=1)) / (2 * dx)
    return u, v


In [17]:
# Define parameters for the simulation
nx, ny = 100, 100  # Grid size
dx, dy = 1.0, 1.0  # Differential step size
dt = 0.01  # Time step size
nt = 100  # Number of time steps to include in the animation

# Define the wave numbers for the initial condition
kx = 2 * np.pi / (nx * dx)  # Wave number in x-direction
ky = 2 * np.pi / (ny * dy)  # Wave number in y-direction

# Initialize the vorticity with a sinusoidal function
X, Y = np.meshgrid(np.arange(nx), np.arange(ny))
omega = np.sin(kx * X) * np.sin(ky * Y) + np.sin(2 * kx * X) * np.sin(2 * ky * Y) + 0.1 * np.cos(kx * (X - 1)) * np.sin(ky * (Y + 1))

# Setup the figure for animation
fig, ax = plt.subplots(figsize=(5, 5))
img = ax.imshow(omega, cmap='RdYlBu', extent=(0, nx*dx, ny*dy, 0))
plt.colorbar(img, ax=ax)
ax.set_title('Vorticity Evolution')

# Initialize function for the animation
def init():
    img.set_data(np.zeros((nx, ny)))
    return img,

# Animation update function
def update(frame):
    global omega, u, v
    psi = solve_poisson_fft(omega, dx, dy)
    u, v = velocity_from_streamfunction(psi, dx, dy)
    omega = forward_euler(omega, u, v, nx, ny, dx, dy, dt)
    img.set_data(omega)
    img.set_clim(omega.min(), omega.max())
    return img,

# Create the animation
anim = FuncAnimation(fig, update, frames=nt, init_func=init, blit=True)

# Save the animation to a file
anim_file = 'vorticity_evolution_euler.mp4'
anim.save(anim_file, fps=15, extra_args=['-vcodec', 'libx264'])

# Close the plot to free up memory
plt.close(fig)


In [18]:
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve

# Update the forward Euler function to Crank-Nicolson for the vorticity field
def crank_nicolson_vorticity(omega, u, v, nx, ny, dx, dy, dt):
    # Initialize the coefficients for the linear system
    diagonal = np.zeros(nx * ny)
    lower_diagonal = np.zeros(nx * ny - 1)
    upper_diagonal = np.zeros(nx * ny - 1)
    b = np.zeros(nx * ny)

    for i in range(nx):
        for j in range(ny):
            # Linear index for the current grid point
            index = i * ny + j

            # Fill in the b vector with the known values
            dwdx = (omega[(i+1)%nx, j] - omega[(i-1)%nx, j]) / (2*dx)
            dwdy = (omega[i, (j+1)%ny] - omega[i, (j-1)%ny]) / (2*dy)
            b[index] = omega[i, j] - dt / 2 * (u[i, j] * dwdx + v[i, j] * dwdy)

            # Setup the main diagonal of the system matrix
            diagonal[index] = 1

            # Setup the lower and upper diagonals of the system matrix
            if j > 0:
                lower_diagonal[index - 1] = dt / 2 * (u[i, j-1] / (2*dx) + v[i, j-1] / (2*dy))
            if j < ny - 1:
                upper_diagonal[index] = -dt / 2 * (u[i, j+1] / (2*dx) + v[i, j+1] / (2*dy))

    # Construct the sparse matrix A
    A = diags(
        diagonals=[diagonal, lower_diagonal, upper_diagonal, lower_diagonal, upper_diagonal],
        offsets=[0, -1, 1, -ny, ny],
        shape=(nx*ny, nx*ny),
        format="csr"
    )

    # Solve the linear system
    omega_next_flat = spsolve(A, b)
    
    # Reshape the solution vector to the 2D field
    omega_next = omega_next_flat.reshape((nx, ny))

    return omega_next

# Update the time-stepping loop to use the Crank-Nicolson method
for n in range(nt):
    # Solve for the stream function using the previous vorticity
    psi = solve_poisson_fft(omega, dx, dy)

    # Compute the velocity field from the stream function
    u, v = velocity_from_streamfunction(psi, dx, dy)

    # Update the vorticity field using the Crank-Nicolson method
    omega = crank_nicolson_vorticity(omega, u, v, nx, ny, dx, dy, dt)

# Display a small part of the updated vorticity field as a sanity check
omega[:5, :5]



array([[ 0.00531039,  0.0104234 ,  0.02280646,  0.04264742,  0.06943186],
       [-0.00788863, -0.00087289,  0.01457289,  0.03956426,  0.07375378],
       [-0.05039252, -0.03186331, -0.01701215,  0.00853201,  0.04329077],
       [-0.12830233, -0.07773126, -0.06641697, -0.03845819, -0.00079932],
       [-0.24463207, -0.13531539, -0.13436445, -0.10203695, -0.06065802]])

In [14]:
# Define parameters for the simulation
nx, ny = 100, 100  # Grid size
dx, dy = 1.0, 1.0  # Differential step size
dt = 0.01  # Time step size
nt = 400  # Number of time steps to include in the animation

# Define the wave numbers for the initial condition
kx = 2 * np.pi / (nx * dx)  # Wave number in x-direction
ky = 2 * np.pi / (ny * dy)  # Wave number in y-direction

# Initialize the vorticity with a sinusoidal function
X, Y = np.meshgrid(np.arange(nx), np.arange(ny))
omega = np.sin(kx * X) * np.sin(ky * Y)

# Setup the figure for animation
fig, ax = plt.subplots(figsize=(5, 5))
img = ax.imshow(omega, cmap='RdYlBu', extent=(0, nx*dx, ny*dy, 0))
plt.colorbar(img, ax=ax)
ax.set_title('Vorticity Evolution')

# Initialize function for the animation
def init():
    img.set_data(np.zeros((nx, ny)))
    return img,

# Animation update function
def update(frame):
    global omega, u, v
    # Solve for the stream function using the previous vorticity
    psi = solve_poisson_fft(omega, dx, dy)

    # Compute the velocity field from the stream function
    u, v = velocity_from_streamfunction(psi, dx, dy)

    # Update the vorticity field using the Crank-Nicolson method
    omega = crank_nicolson_vorticity(omega, u, v, nx, ny, dx, dy, dt)
    
    img.set_data(omega)
    img.set_clim(omega.min(), omega.max())
    return img,

# Create the animation
anim = FuncAnimation(fig, update, frames=nt, init_func=init, blit=True)

# Save the animation to a file
anim_file = 'vorticity_evolution_cn.mp4'
anim.save(anim_file, fps=15, extra_args=['-vcodec', 'libx264'])

# Close the plot to free up memory
plt.close(fig)

C:\Users\igor-\AppData\Local\Temp\ipykernel_7132\1980191747.py:20: RuntimeWarning: overflow encountered in scalar multiply
  b[index] = omega[i, j] - dt / 2 * (u[i, j] * dwdx + v[i, j] * dwdy)
C:\Users\igor-\AppData\Local\Temp\ipykernel_7132\1980191747.py:20: RuntimeWarning: invalid value encountered in scalar add
  b[index] = omega[i, j] - dt / 2 * (u[i, j] * dwdx + v[i, j] * dwdy)
C:\Users\igor-\AppData\Local\Temp\ipykernel_7132\1980191747.py:20: RuntimeWarning: overflow encountered in scalar add
  b[index] = omega[i, j] - dt / 2 * (u[i, j] * dwdx + v[i, j] * dwdy)
c:\Users\igor-\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:293: MatrixRankWarning: Matrix is exactly singular
  warn("Matrix is exactly singular", MatrixRankWarning)
